In [1]:
from ipywidgets import interact
from brian2 import * #type: ignore
import NeuronEquations
import BrianHF
from ipywidgets import interact
import numpy as np

In [2]:
# @interact(tau=(0.01,5, 0.01), beta=(0.1, 5.0, 0.05), Wi=(0.0, 20.0, 0.05), Wk=(-10.0, 0.0, 0.05), vt=(0.1, 1.0, 0.1), vr=(0.0, 1.1, 0.1))
# def simulate_neurons(tau=0.2, vt=0.1, vr=0.0, beta=0.5, Wi=6.15, Wk=-10):
@interact(tau=(0.01,5, 0.01), beta=(0.1, 5.0, 0.05), Wi=(0.0, 20.0, 0.01), Wk=(-20.0, 0.0, 0.01), vt=(0.1, 1.0, 0.1), vr=(0.0, 1.1, 0.1))
def simulate_neurons(tau=0.2, vt=0.1, vr=0.0, beta=0.5, Wi=6.0, Wk=-3.0):
    start_scope()
    
    defaultclock.dt = 0.5*ms

    grid_size_x = 5 # Number of neurons in the x direction
    grid_size_y = 5
    N_Neurons = grid_size_x * grid_size_y # Number of neurons\
        
        
    # Neuron Equation
    Eqs_Neurons = NeuronEquations.EQ_SCM_IF

    # Neuron Parameters
    Neuron_Params = {'tau': tau*ms, 'vt': vt, 'vr': vr, 'P': 0, 'incoming_spikes': 0, 'method': 'exact'}
    # Synapse Parameters
    Syn_Params = {'Num_Neighbours' : 8, 'beta': beta, 'Wi': Wi, 'Wk': Wk}
    Num_Neighbours = Syn_Params['Num_Neighbours']

    input = []
    # Input generation
    for i in range (5):
        input.extend(range(i, N_Neurons, 5))
    times = []
    for i in range(5):
        times.extend([i]*5)
    
    G_Spikes = SpikeGeneratorGroup(N_Neurons, input, times*ms)



    # indices = array([4, 5, 3, 4, 5])
    # times = array([0, 0.5, 0.5, 1.0, 1.2])*ms
    # G_Spikes = SpikeGeneratorGroup(N_Neurons, indices, times)
    

    
    # indices = np.concatenate((arange(N_Neurons), arange(5), arange(7,11)))
    # times = np.concatenate(([1]*N_Neurons, [2]*5, [3]*4))*ms
    # G_Spikes = SpikeGeneratorGroup(N_Neurons, indices, times)

    neuronsGrid = NeuronGroup(N_Neurons, Eqs_Neurons, threshold='v>vt',
                            reset='''
                            v = vr
                            incoming_spikes_post = 0
                            ''',
                            refractory='0.01*ms',
                            events={'P_ON': 'v > vt', 'P_OFF': '(timestep(t - lastspike, dt) > timestep(dt, dt) and v <= vt)'},
                            namespace=Neuron_Params)
    
    neuronsGrid.set_event_schedule('P_ON', when = 'after_thresholds')
    neuronsGrid.run_on_event('P_ON', 'P = 1' , when = 'after_thresholds')
    neuronsGrid.set_event_schedule('P_OFF', when = 'groups')
    neuronsGrid.run_on_event('P_OFF', 'P = 0', when = 'groups')

    # Generate x and y values for each neuron
    x_values = np.repeat(np.arange(grid_size_x), grid_size_y)
    y_values = np.tile(np.arange(grid_size_y), grid_size_x)

    # Assign x, y, and z values to each neuron
    neuronsGrid.X = x_values
    neuronsGrid.Y = y_values

    # Creating the synapses
    Syn_Input_Neurons = Synapses(G_Spikes, neuronsGrid,
                                 'beta : 1 (constant)',
                                 on_pre='''
                                 ExtIn_post = beta
                                 ''',
                                 namespace=Syn_Params)
    
    # Syn_Exc_Neurons = Synapses(neuronsGrid, neuronsGrid,
    #                            '''
    #                            Wi : 1
    #                            ''',
    #                            on_pre={'pre':'incoming_spikes_post += 1; Exc_post = Wi'}, 
    #                            namespace=Syn_Params)

    Syn_Neurons_Neurons = Synapses(neuronsGrid, neuronsGrid,
                               '''
                               Wi : 1
                               Wk : 1
                               ''',
                               on_pre={'pre': 'incoming_spikes_post += 1; Exc_pre = Wi',
                                   'pre_2' : 'Inh_post = P_post * (Wk/incoming_spikes_post)'},
                               namespace=Syn_Params)

    
    # SYNAPSE CONNECTIONS:
    # Connect the first synapses from input to neuronsGrid on a 1 to 1 basis
    Syn_Input_Neurons.connect(j = 'i')
    Syn_Input_Neurons.beta = beta
    
    # Syn_Exc_Neurons.connect(j = 'i')
    # Syn_Exc_Neurons.Wi = Wi


    # Unzip the pairs into two lists
    indexes_i, indexes_j = BrianHF.calculate_ChebyshevNeighbours(neuronsGrid, Num_Neighbours)
    # Connect the last group of synapses from a neuron to its neighbors
    Syn_Neurons_Neurons.connect(i=indexes_i, j=indexes_j)
    Syn_Neurons_Neurons.Wk = Wk
    Syn_Neurons_Neurons.Wi = Wi

    # SETTING UP MONITORS:
    # Monitor the spikes
    SpikeMon_Input = SpikeMonitor(G_Spikes)
    SpikeMon_Neurons = SpikeMonitor(neuronsGrid)
    P_ON_Mon = EventMonitor(neuronsGrid, 'P_ON')
    P_OFF_Mon = EventMonitor(neuronsGrid, 'P_OFF')

    # Monitor the state variables
    StateMon_Neurons = StateMonitor(neuronsGrid, ['v', 'In', 'ExtIn', 'SystIn', 'P', 'Exc', 'Inh'], record=True)
    
    
    
    # RUNNING THE SIMULATION:
    # display(scheduling_summary())
    BrianLogger.log_level_error()
    run_time = 20*ms
    run(run_time, report='text')

    # Plot the results
    figure(figsize=(15, 12))
    subplot(311)
    plot(SpikeMon_Input.t/ms, SpikeMon_Input.i, '.k')
    plt.xlim(-0.5, run_time/ms+0.5)
    ylabel('Neuron index')
    title('Input spikes')

    # Plot the results
    subplot(312)
    plot(SpikeMon_Neurons.t/ms, SpikeMon_Neurons.i, '.k')
    plt.xlim(-0.5, run_time/ms+0.5)
    ylabel('Neuron index')
    title('Neuron Grid spikes')

    # Adjust the spacing between the graphs
    plt.subplots_adjust(hspace=0.8)  # Adjust the vertical spacing between the plots
    
    # subplot(313)
    # plot(P_OFF_Mon.t/ms, P_OFF_Mon.i, '.k')
    # plot(P_ON_Mon.t/ms, P_ON_Mon.i, 'xr')
    # xlabel('Time (ms)')
    # ylabel('Neuron index')
    # title('P_ON and P_OFF events')
    # legend(['P_OFF', 'P_ON'])
    show()

    # BrianHF.visualise_neurons_states(StateMon_Neurons, [3], ['v', 'In', 'SystIn', 'ExtIn'], overlap = False, vt = Neuron_Params['vt'])
    # BrianHF.visualise_neurons_states(StateMon_Neurons, [3], ['SystIn', 'P', 'Inh', 'Exc'], overlap = False, vt = Neuron_Params['vt'])
    
    # BrianHF.visualise_neurons_states(StateMon_Neurons, [4], ['v', 'In', 'SystIn', 'ExtIn'], overlap = False, vt = Neuron_Params['vt'])
    # BrianHF.visualise_neurons_states(StateMon_Neurons, [4], ['SystIn', 'P', 'Inh', 'Exc'], overlap = False, vt = Neuron_Params['vt'])
    
    # BrianHF.visualise_neurons_states(StateMon_Neurons, [5], ['v', 'In', 'SystIn', 'ExtIn'], overlap = True, vt = Neuron_Params['vt'])
    # BrianHF.visualise_neurons_states(StateMon_Neurons, [5], ['SystIn', 'P', 'Inh', 'Exc'], overlap = True, vt = Neuron_Params['vt'])

    BrianHF.visualise_states(StateMon_Neurons, [3, 4, 5], ['v', 'In', 'SystIn', 'ExtIn'], overlap = True)
    BrianHF.visualise_states(StateMon_Neurons, [3, 4, 5], ['SystIn', 'P', 'Inh', 'Exc'], overlap = True)
    
    BrianHF.visualise_interSpikeInterval(SpikeMon_Neurons, [3,4])

interactive(children=(FloatSlider(value=0.2, description='tau', max=5.0, min=0.01, step=0.01), FloatSlider(val…